In [1]:
import pandas as pd
data = pd.read_csv(r"C:\Users\shahv\Downloads\abcnews-date-text-small.csv", error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

C:\Users\shahv\AppData\Local\Temp\ipykernel_7760\2510013797.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(r"C:\Users\shahv\Downloads\abcnews-date-text-small.csv", error_bad_lines=False);


In [2]:
print(len(documents))
print(documents[:5])

19999
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


# Data Pre-processing
Tokenization
Words that have fewer than 3 characters are removed.
stopwords 
lemmatized/stemmed

In [3]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk

In [4]:
#from nltk.stem.porter import stemmer

In [5]:
def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

In [7]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2:
            result.append(lemmatize_stemming(token))
    return result

# Select a document to preview after preprocessing.

In [8]:
doc_sample = documents[documents['index'] == 8500].values[0][0]
print('original document: ')
words = []
    for word in doc_sample.split(' '):
        words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['nz', 'suspends', 'loose', 'tobacco', 'sales']


 tokenized and lemmatized document: 
['suspend', 'loose', 'tobacco', 'sales']


Preprocess the headline text, saving the results as ‘processed_docs’

In [9]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0         [aba, decide, community, broadcast, licence]
1                    [act, witness, aware, defamation]
2           [call, infrastructure, protection, summit]
3                [air, staff, aust, strike, pay, rise]
4        [air, strike, affect, australian, travellers]
5               [ambitious, olsson, win, triple, jump]
6               [antic, delight, record, break, barca]
7    [aussie, qualifier, stosur, waste, memphis, ma...
8             [aust, address, security, council, iraq]
9               [australia, lock, war, timetable, opp]
Name: headline_text, dtype: object

# Word embedding
Bag of Words on the Data
Create a dictionary from ‘processed_docs’ containing the number of times a word appears in the training set.

In [10]:
#Creating a vocabulary
dictionary = gensim.corpora.Dictionary(processed_docs)

In [11]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 aba
1 broadcast
2 community
3 decide
4 licence
5 act
6 aware
7 defamation
8 witness
9 call
10 infrastructure


Gensim filter_extremes

Filter out tokens that appear in

    less than 15 documents (absolute number) or
    more than 0.5 documents (fraction of total corpus size, not absolute number).
    after the above two steps, keep only the first 100000 most frequent tokens.

In [14]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=10000)

# Gensim doc2bow
For each document we create a dictionary reporting how many
words and how many times those words appear. Save this to ‘bow_corpus’, then check our selected document earlier.

In [15]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(114, 1), (176, 1), (207, 1), (425, 1), (580, 1)]

Preview Bag Of Words for our sample preprocessed document.

In [16]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 114 ("govt") appears 1 time.
Word 176 ("group") appears 1 time.
Word 207 ("vote") appears 1 time.
Word 425 ("local") appears 1 time.
Word 580 ("want") appears 1 time.


# TF-IDF

Create tf-idf model object using models.TfidfModel on ‘bow_corpus’ and save it to ‘tfidf’, then apply transformation to the entire corpus and call it ‘corpus_tfidf’. Finally we preview TF-IDF scores for our first document.

In [17]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.49655634360283785), (1, 0.5786218695719839), (2, 0.6470150923130019)]


# Running LDA using Bag of Words
Train our lda model using gensim.models.LdaMulticore and save it to ‘lda_model’

In [18]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

For each topic, we will explore the words occuring in that topic and its relative weight.

In [19]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.017*"police" + 0.017*"cup" + 0.017*"probe" + 0.014*"world" + 0.013*"get" + 0.012*"day" + 0.011*"south" + 0.011*"bid" + 0.010*"home" + 0.010*"crash"
Topic: 1 
Words: 0.039*"govt" + 0.018*"pay" + 0.016*"urge" + 0.014*"iraqi" + 0.014*"lead" + 0.012*"new" + 0.012*"council" + 0.011*"service" + 0.010*"nsw" + 0.010*"win"
Topic: 2 
Words: 0.029*"sars" + 0.017*"welcome" + 0.013*"decision" + 0.012*"new" + 0.011*"fight" + 0.011*"force" + 0.011*"urge" + 0.011*"cut" + 0.010*"share" + 0.009*"talk"
Topic: 3 
Words: 0.023*"meet" + 0.022*"fund" + 0.020*"concern" + 0.019*"water" + 0.018*"boost" + 0.017*"seek" + 0.015*"council" + 0.015*"air" + 0.013*"hold" + 0.013*"rule"
Topic: 4 
Words: 0.041*"man" + 0.031*"face" + 0.029*"police" + 0.028*"kill" + 0.026*"charge" + 0.016*"war" + 0.014*"court" + 0.013*"murder" + 0.013*"iraq" + 0.012*"car"
Topic: 5 
Words: 0.037*"plan" + 0.018*"win" + 0.014*"vic" + 0.014*"end" + 0.012*"sydney" + 0.012*"govt" + 0.012*"search" + 0.012*"man" + 0.010*"miss" +

# Running LDA using TF-IDF

In [20]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)


In [21]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.013*"water" + 0.012*"iraq" + 0.011*"say" + 0.010*"set" + 0.010*"call" + 0.009*"govt" + 0.008*"troop" + 0.008*"year" + 0.007*"vic" + 0.007*"school"
Topic: 1 Word: 0.013*"win" + 0.011*"air" + 0.011*"aceh" + 0.008*"drought" + 0.008*"kill" + 0.008*"lead" + 0.008*"test" + 0.007*"govt" + 0.007*"put" + 0.007*"concern"
Topic: 2 Word: 0.011*"hit" + 0.008*"save" + 0.008*"cut" + 0.007*"mayor" + 0.007*"french" + 0.007*"decision" + 0.007*"new" + 0.007*"change" + 0.006*"brumbies" + 0.006*"court"
Topic: 3 Word: 0.011*"war" + 0.010*"govt" + 0.007*"lose" + 0.007*"australia" + 0.007*"bid" + 0.007*"nsw" + 0.007*"end" + 0.007*"council" + 0.006*"challenge" + 0.006*"claim"
Topic: 4 Word: 0.009*"police" + 0.008*"death" + 0.008*"korea" + 0.008*"rescue" + 0.007*"defend" + 0.007*"north" + 0.007*"warn" + 0.007*"medicare" + 0.007*"doctor" + 0.007*"govt"
Topic: 5 Word: 0.014*"man" + 0.009*"return" + 0.008*"hold" + 0.008*"police" + 0.008*"kill" + 0.007*"industry" + 0.007*"decision" + 0.007*"abuse" 

# Performance evaluation by classifying sample document using LDA Bag of Words model

We will check where our test document would be classified.

In [22]:
processed_docs[4310]

['ratepayers', 'group', 'want', 'compulsory', 'local', 'govt', 'vote']

In [23]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.5576344132423401	 
Topic: 0.017*"toll" + 0.016*"budget" + 0.015*"rise" + 0.013*"race" + 0.013*"jail" + 0.012*"open" + 0.012*"spark" + 0.011*"ban" + 0.009*"death" + 0.009*"study"

Score: 0.30899739265441895	 
Topic: 0.039*"govt" + 0.018*"pay" + 0.016*"urge" + 0.014*"iraqi" + 0.014*"lead" + 0.012*"new" + 0.012*"council" + 0.011*"service" + 0.010*"nsw" + 0.010*"win"

Score: 0.01667516678571701	 
Topic: 0.029*"sars" + 0.017*"welcome" + 0.013*"decision" + 0.012*"new" + 0.011*"fight" + 0.011*"force" + 0.011*"urge" + 0.011*"cut" + 0.010*"share" + 0.009*"talk"

Score: 0.01667369343340397	 
Topic: 0.024*"iraq" + 0.017*"hear" + 0.016*"troop" + 0.012*"bali" + 0.011*"deny" + 0.011*"call" + 0.011*"talk" + 0.010*"say" + 0.009*"bomb" + 0.009*"baghdad"

Score: 0.016671519726514816	 
Topic: 0.043*"new" + 0.026*"police" + 0.017*"case" + 0.013*"hit" + 0.012*"return" + 0.010*"govt" + 0.009*"appeal" + 0.008*"second" + 0.008*"sars" + 0.008*"plan"

Score: 0.016670450568199158	 
Topic: 0.037*"plan" 

# Performance evaluation by classifying sample document using LDA TF-IDF model.

In [24]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.4866695702075958	 
Topic: 0.014*"man" + 0.009*"return" + 0.008*"hold" + 0.008*"police" + 0.008*"kill" + 0.007*"industry" + 0.007*"decision" + 0.007*"abuse" + 0.006*"child" + 0.006*"future"

Score: 0.37995463609695435	 
Topic: 0.009*"police" + 0.008*"death" + 0.008*"korea" + 0.008*"rescue" + 0.007*"defend" + 0.007*"north" + 0.007*"warn" + 0.007*"medicare" + 0.007*"doctor" + 0.007*"govt"

Score: 0.01667529158294201	 
Topic: 0.015*"court" + 0.012*"claim" + 0.011*"face" + 0.011*"take" + 0.010*"charge" + 0.009*"get" + 0.009*"urge" + 0.008*"sign" + 0.007*"fight" + 0.006*"green"

Score: 0.0166742242872715	 
Topic: 0.011*"war" + 0.010*"govt" + 0.007*"lose" + 0.007*"australia" + 0.007*"bid" + 0.007*"nsw" + 0.007*"end" + 0.007*"council" + 0.006*"challenge" + 0.006*"claim"

Score: 0.016673095524311066	 
Topic: 0.017*"police" + 0.012*"man" + 0.010*"charge" + 0.009*"fund" + 0.009*"seek" + 0.009*"consider" + 0.008*"new" + 0.008*"help" + 0.008*"job" + 0.007*"hospital"

Score: 0.016671609133

# Testing model on unseen document

In [25]:
unseen_document = 'stock market crashed to its lowest 52 week'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.4002724885940552	 Topic: 0.017*"police" + 0.017*"cup" + 0.017*"probe" + 0.014*"world" + 0.013*"get"
Score: 0.29640597105026245	 Topic: 0.023*"claim" + 0.019*"reject" + 0.017*"consider" + 0.014*"court" + 0.014*"baghdad"
Score: 0.16329114139080048	 Topic: 0.037*"plan" + 0.018*"win" + 0.014*"vic" + 0.014*"end" + 0.012*"sydney"
Score: 0.02000841125845909	 Topic: 0.024*"iraq" + 0.017*"hear" + 0.016*"troop" + 0.012*"bali" + 0.011*"deny"
Score: 0.020007459446787834	 Topic: 0.041*"man" + 0.031*"face" + 0.029*"police" + 0.028*"kill" + 0.026*"charge"
Score: 0.020003758370876312	 Topic: 0.039*"govt" + 0.018*"pay" + 0.016*"urge" + 0.014*"iraqi" + 0.014*"lead"
Score: 0.020003097131848335	 Topic: 0.017*"toll" + 0.016*"budget" + 0.015*"rise" + 0.013*"race" + 0.013*"jail"
Score: 0.020002862438559532	 Topic: 0.023*"meet" + 0.022*"fund" + 0.020*"concern" + 0.019*"water" + 0.018*"boost"
Score: 0.02000257931649685	 Topic: 0.043*"new" + 0.026*"police" + 0.017*"case" + 0.013*"hit" + 0.012*"return"
